# Reduce PDS data

To _reduce_ PDS may be different across different datasets/product-types

Here, we are going to work with MEX/HRSC data.

In [5]:
dataset = 'hrsc'

In [9]:
from gpt.helpers import collection

from importlib import reload
reload(collection)

<module 'gpt.helpers.collection' from '/Users/chbrandt/Coisas/repos/GMAP/gpt/gpt/helpers/collection.py'>

In [10]:
ls ../01_search-download_from_ode-pds/

01_search_data_products.ipynb    download_hrsc.geojson
02_download_data_products.ipynb  search_ctx.geojson
data/                            search_hirise.geojson
download_ctx.geojson             search_hrsc.geojson
download_hirise.geojson


In [11]:
from pathlib import Path

db_path = Path(f'../01_search-download_from_ode-pds/download_{dataset}.geojson')

archive = collection.from_geojson(db_path.as_posix())

In [12]:
archive

                                            geometry BB_georeferenced  \
0  MULTIPOLYGON (((-0.21680 3.84280, -0.21600 3.9...             True   
1  MULTIPOLYGON (((-0.21680 3.84280, -0.21600 3.9...             True   
2  POLYGON ((-1.73400 -11.89400, -1.73800 -11.644...             True   
3  POLYGON ((-1.73400 -11.89400, -1.73800 -11.644...             True   

  Center_georeferenced Center_latitude Center_longitude  \
0                 True         -1.7901           0.2893   
1                 True         -1.7901           0.2893   
2                 True         -4.8102         358.8204   
3                 True         -4.8102         358.8204   

                            Data_Set_Id Easternmost_longitude  \
0  MEX-M-HRSC-5-REFDR-MAPPROJECTED-V3.0               359.784   
1  MEX-M-HRSC-5-REFDR-MAPPROJECTED-V3.0               359.784   
2  MEX-M-HRSC-5-REFDR-MAPPROJECTED-V3.0               359.921   
3  MEX-M-HRSC-5-REFDR-MAPPROJECTED-V3.0               359.921   

            